In [1]:
!pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
api_key="pcsk_4w9PjE_SD8Ct9gVEvYQebvJq9wTvxVBqB7bFPdCJpBmUXSWqaK78mp5prPrWNi58skV6XJ"

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
#setup pinecone

import os

from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# initialize Pinecone client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [5]:
# Now that the index is created, we can use it.

index = pc.Index(index_name)
index

d:\Projects\Hybrid-Search-Engine-using-Langchain\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#from langchain_openai import OpenAIEmbeddings
#embeddings = OpenAIEmbeddings()
embeddings 

d:\Projects\Hybrid-Search-Engine-using-Langchain\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
## To encode the text to sparse values you can either choose SPLADE or BM25. For out of domain tasks we recommend using BM25.
# uses TFIDF encoder

from pinecone_text.sparse import BM25Encoder

# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [14]:
sentences=[
    "In 2023, I visited Paris",
     "In 2022, I visited New York",
     "In 2021, I visited Tokyo",
]

#tfdif values on these sentences
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

#load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 3990.77it/s]


In [15]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000021DD5EFF2F0>, index=<pinecone.db_data.index.Index object at 0x0000021DFA5E0440>)

In [27]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
     "In 2022, I visited New York",
     "In 2021, I visited Tokyo",
]
)

100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


In [30]:
retriever.invoke("what city did I visit most recent")

[Document(metadata={'score': 0.239538237}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.226800025}, page_content='In 2021, I visited Tokyo'),
 Document(metadata={'score': 0.207687438}, page_content='In 2023, I visited Paris')]